In [1]:
# Attempt to train an mnist model and distill it into a smaller model
# conclusion, distillation is a scam, at least for mnist
# seems to also be a scam for CIFAR

In [2]:
%pylab inline
import os
import time
import random
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = [x/255.0 for x in [x_train, x_test]]
y_train, y_test = [tf.keras.utils.to_categorical(x) for x in [y_train, y_test]]

Populating the interactive namespace from numpy and matplotlib


In [3]:
def get_m(scale=64, loss='categorical_crossentropy'):
    x = in1 = Input((32,32,3))

    x = Conv2D(scale, (3,3), padding='same', strides=(2,2))(x)
    x = ELU()(x)

    x = Conv2D(2*scale, (3,3), padding='same', strides=(2,2))(x)
    x = ELU()(x)

    x = Flatten()(x)
    x = Dense(128)(x)
    x = ELU()(x)
    x = Dense(10, activation='softmax')(x)

    m = Model(in1, x)
    m.compile('adam', loss, metrics=['accuracy'])
    return m

In [4]:
m = get_m()
m.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 64)        1792      
_________________________________________________________________
elu (ELU)                    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
elu_1 (ELU)                  (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               104870

In [5]:
m.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=5)
rv = m.predict(x_train, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 14s 289us/sample - loss: 1.7983 - accuracy: 0.3580 - val_loss: 1.5966 - val_accuracy: 0.4453
Epoch 2/5
50000/50000 [==============================] - 14s 288us/sample - loss: 1.5088 - accuracy: 0.4696 - val_loss: 1.4716 - val_accuracy: 0.4775
Epoch 3/5
50000/50000 [==============================] - 14s 284us/sample - loss: 1.3744 - accuracy: 0.5152 - val_loss: 1.4038 - val_accuracy: 0.5014
Epoch 4/5
50000/50000 [==============================] - 14s 285us/sample - loss: 1.2536 - accuracy: 0.5579 - val_loss: 1.3376 - val_accuracy: 0.5258
Epoch 5/5
50000/50000 [==============================] - 6s 125us/sample


In [6]:
sm1 = get_m(4)
sm2 = get_m(4)
sm1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 4)         112       
_________________________________________________________________
elu_3 (ELU)                  (None, 16, 16, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           296       
_________________________________________________________________
elu_4 (ELU)                  (None, 8, 8, 8)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               6566

In [7]:
sm1.fit(x_train, rv, validation_data=(x_test, y_test), batch_size=128, epochs=3)

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 4s 71us/sample - loss: 0.8286 - accuracy: 0.3826 - val_loss: 1.7425 - val_accuracy: 0.3817
Epoch 2/3
50000/50000 [==============================] - 3s 61us/sample - loss: 0.7156 - accuracy: 0.4965 - val_loss: 1.6442 - val_accuracy: 0.4198
Epoch 3/3
50000/50000 [==============================] - 3s 63us/sample - loss: 0.6614 - accuracy: 0.5613 - val_loss: 1.6090 - val_accuracy: 0.4279


In [8]:
sm1.evaluate(x_train, y_train)

50000/50000 [==============================] - 2s 37us/sample - loss: 1.5750 - accuracy: 0.4441


[1.575022049560547, 0.44412]

In [9]:
sm2.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=3)

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 3s 70us/sample - loss: 1.8284 - accuracy: 0.3523 - val_loss: 1.7143 - val_accuracy: 0.3960
Epoch 2/3
50000/50000 [==============================] - 3s 62us/sample - loss: 1.6540 - accuracy: 0.4218 - val_loss: 1.5993 - val_accuracy: 0.4405
Epoch 3/3
50000/50000 [==============================] - 3s 64us/sample - loss: 1.5471 - accuracy: 0.4557 - val_loss: 1.5292 - val_accuracy: 0.4618
